In [2]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np

NanoAODSchema.warn_missing_crossrefs = False

In [1]:
fname = "root://eos.cms.rcac.purdue.edu//store/mc/RunIISummer20UL18NanoAODv9/GluGluHToMuMu_M125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2810000/C4DAB63C-E2A1-A541-93A8-3F46315E362C.root"
# fname = "root://eos.cms.rcac.purdue.edu//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/40000/ECB51118-0153-2F40-BB6D-0204F0EE98C2.root"

events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
).events()


# events = NanoEventsFactory.from_root(
#     {fname : "Events"},
#     schemaclass=NanoAODSchema,
#     metadata={"dataset": "DYJets"},
#     delayed=False,
# ).events()


# filtered_events =  events.Muon.pt[events.HLT.IsoMu24]
# leading_muPt = ak.to_numpy(
#     ak.flatten(filtered_events[:,:1]).compute()
# )
# nonFiltered_leading_muPt = ak.to_numpy(ak.flatten(events.Muon.pt[:,:1]).compute())
# xmax= 80
# plt.xlim([5, xmax])
# plt.hist(leading_muPt, bins=np.linspace(5, xmax, 100),histtype=u'step', label="HLT")
# plt.hist(nonFiltered_leading_muPt, bins=np.linspace(5, xmax, 100),histtype=u'step', label="pre-HLT")
# plt.axvline(x=24, color='red', ls='--')
# plt.axvline(x=26, color='teal', ls='--')
# plt.xlabel("Leading Muon pT")
# plt.title("ggH powheg histogram")

# plt.legend(loc='best')
# plt.show()
# plt.savefig("ggH_turnOn.png")

NameError: name 'NanoEventsFactory' is not defined

In [10]:
IsoMu24_arr = events.HLT.IsoMu24.compute()
print(len(IsoMu24_arr))
ak.num(IsoMu24_arr, axis=0)

999000


array(999000)

In [13]:
IsoMu24_arr = events.HLT.IsoMu24
isoMu_filterbit = 2
mu_id = 13
thresholds = [23,23.5,23.8,24,24.2,24.5]
total_events = ak.num(IsoMu24_arr, axis=0)
for threshold in thresholds:
    IsoMu24_muons = (events.TrigObj.id == mu_id) &  \
        ((events.TrigObj.filterBits & isoMu_filterbit) == isoMu_filterbit) & \
        (events.TrigObj.pt > threshold) # considering wildcard scenrios where exact 24 pt cut ain't doing it
    isohlt_overall = ak.sum(IsoMu24_muons, axis=1) > 0
    evnt_mismatchrate = ak.sum(isohlt_overall != IsoMu24_arr) / total_events
    print(f"trigger threshold {threshold} evnt_mismatchrate rate: {evnt_mismatchrate.compute()}")

print(f"total HLT events: {total_events.compute()}")

trigger threshold 23 evnt_mismatchrate rate: 0.006471471471471472
trigger threshold 23.5 evnt_mismatchrate rate: 0.004608608608608609
trigger threshold 23.8 evnt_mismatchrate rate: 0.0034394394394394396
trigger threshold 24 evnt_mismatchrate rate: 0.0026206206206206206
trigger threshold 24.2 evnt_mismatchrate rate: 0.0033503503503503505
trigger threshold 24.5 evnt_mismatchrate rate: 0.004508508508508508
total HLT events: 999000


In [34]:
isoMu_filterbit = 2
mu_id = 13
threshold =24 # 24 pT cut
IsoMu24_muons = (events.TrigObj.id == mu_id) &  \
        ((events.TrigObj.filterBits & isoMu_filterbit) == isoMu_filterbit) & \
        (events.TrigObj.pt > threshold)
#check the first two leading muons match any of the HLT trigger objs. if neither match, reject event
muons = ak.pad_none(events.Muon, 2)
mu1 = muons[:,0]
mu2 = muons[:,1]
d_r1 = mu1.delta_r(events.TrigObj[IsoMu24_muons]) < 0.1
d_r1 = ak.sum(d_r1, axis=1)
d_r1 = ak.fill_none(d_r1, value=False)
d_r2 = mu2.delta_r(events.TrigObj[IsoMu24_muons]) < 0.1
d_r2 =  ak.sum(d_r2, axis=1)
d_r2 = ak.fill_none(d_r2, value=False)

trigger_match = d_r1 | d_r2

In [38]:
# d_r1.compute()
# d_r2.compute()
muons.pt.compute()

<Array [[49.8, None], [...], ..., [33.6, None]] type='999000 * var * ?float...'>

In [36]:
# ak.sum(d_r1, axis=1).compute()
# ak.sum(d_r2, axis=1).compute()
trigger_match.compute()

<Array [1, 1, 1, 1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1, 1, 1] type='999000 * int64'>

In [28]:
# HLT_objs = events.TrigObj[IsoMu24_muons].compute()
# ak.num(HLT_objs, axis=1)
mu1[1].delta_r(HLT_objs[1]).compute()

TypeError: ak.Record objects are not allowed in this function

This error occurred while calling

    ak.to_layout(
        <Record-typetracer {dxy: ##, dxyErr: ##, ...} type='Muon[dxy: float...'>
        allow_record = False
        regulararray = False
        primitive_policy = 'error'
    )

In [31]:
mu1.compute()

KeyboardInterrupt: 

In [3]:
muHLT = (events.TrigObj.filterBits & 8 ) == 8
muHLT_passed_events = ak.sum(muHLT.compute(),axis=1) > 0
IsoMu24_events = events.HLT.IsoMu24.compute()

In [4]:
mulHLT_passed_events = ak.sum(muHLT.compute(),axis=1) > 0

In [5]:
mulHLT_passed_events

<Array [True, True, True, True, ..., True, True, True] type='999000 * bool'>

In [6]:
IsoMu24_arr = events.HLT.IsoMu24.compute()

In [20]:
ak.sum(mulHLT_passed_events != IsoMu24_arr) / len(mulHLT_passed_events)

0.017076076076076075

In [58]:
wildcard_cut  = (events.TrigObj.pt > 23.5).compute()
wildcard_cut

<Array [[True, True, True], [...], ..., [True]] type='999000 * var * bool[p...'>

In [59]:
wildcardWhlt = wildcard_cut & muHLT.compute()
muHLT_arr = muHLT.compute()

In [60]:
ak.sum(wildcardWhlt != muHLT_arr)

24842

In [61]:
mulHLT_passed_events = ak.sum(wildcardWhlt,axis=1) > 0
mulHLT_passed_events

<Array [True, True, True, True, ..., True, True, True] type='999000 * bool'>

In [70]:
ak.sum(mulHLT_passed_events != IsoMu24_arr) / ak.sum(IsoMu24_arr)

0.005927520400950973

In [ ]:
6995

In [65]:
wildcardWhlt

<Array [[True, False, False], [...], ..., [True]] type='999000 * var * bool'>

In [69]:
events.Muon.pt.compute()

<Array [[49.8], [63.7, 56.7], ..., [33.6]] type='999000 * var * float32[par...'>

In [ ]:
mupt_hlt =  events.Muon.pt[wildcardWhlt== True].compute()
mupt_nonhlt =  events.Muon.pt[wildcardWhlt == False].compute()



xmax= 80
plt.xlim([5, xmax])
plt.hist(mupt_hlt, bins=np.linspace(5, xmax, 100),histtype=u'step', label="HLT")
plt.hist(mupt_nonhlt, bins=np.linspace(5, xmax, 100),histtype=u'step', label="non-HLT")
plt.axvline(x=24, color='red', ls='--')
plt.axvline(x=26, color='teal', ls='--')
plt.xlabel("Leading Muon pT")
plt.title("ggH powheg histogram")

plt.legend(loc='best')
plt.show()

In [76]:
relIso = events.Muon.pfRelIso04_all.compute()

In [78]:
ak.sum(relIso>0.25) /ak.sum(relIso)

5.144145203472276

In [86]:
events.TrigObj[events.TrigObj.id==13].l1charge.compute()

<Array [[1], [-1, 1], ..., [-1, -1], [-1]] type='999000 * var * int32[param...'>

In [90]:
events.TrigObj.fields
# events.TrigObj.id.compute()

['pt',
 'eta',
 'phi',
 'l1pt',
 'l1pt_2',
 'l2pt',
 'id',
 'l1iso',
 'l1charge',
 'filterBits']

In [110]:
isoMu_filterbit = 2
mu_id = 13
bool_filter = (events.TrigObj.id == 13) &  \
    ((events.TrigObj.filterBits & isoMu_filterbit) == isoMu_filterbit) & \
    (events.TrigObj.pt > 24) # considering wildcard scenrios where exact 24 pt cut ain't doing it

In [111]:
hlt_overall = ak.sum(bool_filter, axis=1) >0 

In [112]:
len(IsoMu24_arr)

999000

In [113]:
ak.sum(hlt_overall != IsoMu24_arr).compute()

2618

In [114]:
(ak.sum(hlt_overall != IsoMu24_arr)/len(IsoMu24_arr)).compute()

0.0026206206206206206

In [103]:
(events.TrigObj.id == 13).compute()

<Array [[True, False, False], ..., [True]] type='999000 * var * bool[parame...'>

In [7]:
events.TrigObj.pt.compute()

<Array [[49.7, 55.1, 49.7], [...], ..., [33.6]] type='999000 * var * float3...'>

In [119]:
muTrigPasses = events.TrigObj[bool_filter]
muTrigPasses.delta_r(events.Muon[:,0]).compute()

IndexError: cannot slice ListArray (of length 999000) with array(0): index out of range while attempting to get index 0 (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-28/awkward-cpp/src/cpu-kernels/awkward_ListArray_getitem_next_at.cpp#L21)

This error occurred while attempting to slice

    repr-raised-TypeError

with

    (:, 0)

In [135]:

# mmevents = events[ak.num(events.Muon) > 2]

In [12]:
eventMu =events.Muon.compute()

In [14]:
type(eventMu)

coffea.nanoevents.methods.nanoaod.MuonArray

In [21]:
mevents = events[ak.num(events.Muon) >0][:40]
meventMu =mevents.Muon.compute()

In [19]:
len(meventMu)

40

In [22]:
# mevents.Muon[:,0].delta_r(mevents.Muon).compute()
# events.Muon[:,0].delta_r(events.Muon).compute()
nearest_test = (mevents.Muon.nearest(mevents.Electron)).compute()

In [40]:
nearest_test

<ElectronArray [[None], [None, ...], ..., [None, None]] type='40 * var * ?E...'>

In [45]:
print(type(nearest_test[4,0]))
print(type(nearest_test[6,1]))
# print((nearest_test[4]))
# print((nearest_test[6]))

<class 'coffea.nanoevents.methods.nanoaod.ElectronRecord'>
<class 'coffea.nanoevents.methods.nanoaod.ElectronRecord'>


In [128]:
# events.Electron.delta_r(events.Muon[:,0]).compute()
events.Muon[:,0].delta_r(events.Muon).compute()

dask.awkward<delta-r, npartitions=1>

In [125]:
events.Muon[:,0].compute()

KeyboardInterrupt: 

In [126]:
events.Muon[:,0].fields

['dxy',
 'dxyErr',
 'dxybs',
 'dz',
 'dzErr',
 'eta',
 'ip3d',
 'jetPtRelv2',
 'jetRelIso',
 'mass',
 'miniPFRelIso_all',
 'miniPFRelIso_chg',
 'pfRelIso03_all',
 'pfRelIso03_chg',
 'pfRelIso04_all',
 'phi',
 'pt',
 'ptErr',
 'segmentComp',
 'sip3d',
 'softMva',
 'tkRelIso',
 'tunepRelPt',
 'mvaLowPt',
 'mvaTTH',
 'charge',
 'jetIdx',
 'nStations',
 'nTrackerLayers',
 'pdgId',
 'tightCharge',
 'fsrPhotonIdx',
 'highPtId',
 'highPurity',
 'inTimeMuon',
 'isGlobal',
 'isPFcand',
 'isStandalone',
 'isTracker',
 'jetNDauCharged',
 'looseId',
 'mediumId',
 'mediumPromptId',
 'miniIsoId',
 'multiIsoId',
 'mvaId',
 'mvaLowPtId',
 'pfIsoId',
 'puppiIsoId',
 'softId',
 'softMvaId',
 'tightId',
 'tkIsoId',
 'triggerIdLoose',
 'genPartIdx',
 'genPartFlav',
 'cleanmask',
 'fsrPhotonIdxG',
 'genPartIdxG',
 'jetIdxG']